In [14]:
import os
from openai import OpenAI
from langchain import hub
from langchain.document_loaders import TextLoader
from langchain.schema import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

In [15]:
api_key = os.environ.get("OPENAI_API_KEY")

In [16]:
# Assuming you have three documents in a directory 'docs' with different formats
document_paths = [
    'musk.txt',   # Replace with the actual path to your first document
    'bezos.txt'   # Replace with the actual path to your second document
]
# Initialize an empty list to store all loaded documents
documents = []

In [17]:
for path in document_paths:
    loader = TextLoader(path)
    # Load the document and add it to the list
    documents.extend(loader.load())

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [19]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [20]:
retrieved_docs = retriever.get_relevant_documents(
    "What is elon like as a leader"
)
len(retrieved_docs)
print(retrieved_docs[0].page_content)

a human being and the leader? Well, I don't really know Elon very well. You know, I know his public persona, but I also know you can't know anyone by their public persona. It's impossible. I mean, you may think you do, but I guarantee you don't. So I don't really know you know Elon way better than I do, Lex. But in terms of his judging by the results, he must be a very capable leader. There's no way you could have, you know, Tesla and SpaceX without being a capable leader. It's impossible. Hey, I just, I hope you guys hang out sometimes. Shake hands and sort of have a kind of friendship that would inspire just the entirety of humanity. Because what you're doing is like one of the big grand challenges ahead of humanity. Well, I agree with you. And I think in a lot of these endeavors, we're very like-minded. Yeah. So I think, you know, I'm not saying we're identical, but I think we're very like-minded. And so, I, you know, I love that idea. I go back to sexy pictures on your Instagram.


In [21]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
prompt = hub.pull("rlm/rag-prompt")
print(
    prompt.invoke(
        {"context": "filler context", "question": "filler question"}
    ).to_string()
)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [22]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
for chunk in rag_chain.stream("What is elon like as a leader?"):
    print(chunk, end="", flush=True)

I don't know Elon Musk personally, but based on the context, it seems that he is seen as a capable leader due to the success of Tesla and SpaceX. However, I cannot provide a detailed assessment of his leadership style or qualities.